In [1]:
%load_ext autoreload
%autoreload 2
# %matplotlib widget

In [2]:
import xarray as xr
import numpy as np
import pandas as pd

import ecephys_analyses as ea 

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
xr.set_options(keep_attrs=True)

In [4]:
experiment = "sleep-homeostasis"
nrem = ["N1", "N2"]
light_period_start = "09:00:00"
light_period_end = "21:00:00"


def load_baseline_light_period_nrem(subject):
    condition = "light-period-circadian-match"
    hyp = ea.load_hypnogram(subject, experiment, condition)
    hyp = hyp.keep_states(nrem).keep_between(light_period_start, light_period_end)
    spws = ea.load_spws(subject, experiment, condition)
    spws = spws[hyp.covers_time(spws.start_time)]
    return hyp, spws


def load_early_recovery_nrem(subject):
    condition = "recovery-sleep"
    duration = "01:00:00"
    hyp = ea.load_hypnogram(subject, experiment, condition)
    hyp = hyp.keep_states(nrem).keep_first(duration)
    spws = ea.load_spws(subject, experiment, condition)
    spws = spws[hyp.covers_time(spws.start_time)]
    return hyp, spws


def load_late_recovery_nrem(subject):
    condition = "recovery-sleep"
    duration = "01:00:00"
    hyp = ea.load_hypnogram(subject, experiment, condition)
    hyp = hyp.keep_states(nrem).keep_last(duration)
    spws = ea.load_spws(subject, experiment, condition)
    spws = spws[hyp.covers_time(spws.start_time)]
    return hyp, spws


def load_first2h_recovery_nrem(subject):
    condition = "recovery-sleep"
    duration = "02:00:00"
    hyp = ea.load_hypnogram(subject, experiment, condition)
    hyp = hyp.keep_states(nrem).keep_first(duration)
    spws = ea.load_spws(subject, experiment, condition)
    spws = spws[hyp.covers_time(spws.start_time)]
    return hyp, spws


def load_first2h_recovery_match_nrem(subject, rec_hyp):
    condition = "recovery-sleep-circadian-match"
    duration = "02:00:00"
    hyp = ea.load_hypnogram(subject, experiment, condition)
    hyp = (
        hyp.keep_states(nrem)
        .keep_between(rec_hyp.start_time.min().strftime("%H:%M:%S"), light_period_end)
        .keep_first(duration)
    )
    spws = ea.load_spws(subject, experiment, condition)
    spws = spws[hyp.covers_time(spws.start_time)]
    return hyp, spws

In [5]:
def summarize(subject, condition, hyp, spws):
    count = len(spws)
    duration = hyp.duration.cumsum().max() / pd.to_timedelta(1, 's')
    rate = count / duration
    return dict(subject=subject, condition=condition, count=count, duration_secs=duration, rate_hz=rate)

In [6]:
subjects = ["Segundo", "Valentino", "Doppio", "Alessandro", "Allan"]

subj_dfs = list()
for subject in subjects:
    norm_hyp, norm_spws = load_baseline_light_period_nrem(subject)
    early_hyp, early_spws = load_early_recovery_nrem(subject)
    late_hyp, late_spws = load_late_recovery_nrem(subject)
    rec_hyp, rec_spws = load_first2h_recovery_nrem(subject)
    bsl_hyp, bsl_spws = load_first2h_recovery_match_nrem(subject, rec_hyp)

    subj_dfs.append(
        pd.DataFrame.from_dict(
            [
                summarize(subject, "baseline-light-period-nrem", norm_hyp, norm_spws),
                summarize(subject, "recovery-first2h-match", bsl_hyp, bsl_spws),
                summarize(subject, "recovery-first2h", rec_hyp, rec_spws),
                summarize(subject, "recovery-first1h", early_hyp, early_spws),
                summarize(subject, "recovery-last1h", late_hyp, late_spws),
            ]
        )
    )
    
df = pd.concat(subj_dfs)

In [10]:
df.to_csv('spw-incidence.csv', index=False)